In [ ]:
import warnings
warnings.filterwarnings("ignore")
# warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message=".*Pyarrow will become a required dependency.*")
import pandas as pd
import geopandas as gpd
import numpy as np
import sys # For displaying package versions
import os # For managing directories and file paths if drive is mounted
import random
import json
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
# To reload submodules need to use this magic command to set autoreload on
# This command loads the autoreload extension, enabling the use of the %autoreload magic command.
# This sets autoreload to automatically reload all modules (both modules you've imported and your own modules) before executing a new line. 
# The value "2" means "always reload,"

%load_ext autoreload
%autoreload 2

In [ ]:
!pip install wget
!pip install us
!pip install seaborn
!pip install fpdf2

In [ ]:
from _lodes_data_structure import all_ods
from _lodes_data_structure import all_segparts
from _lodes_data_structure import all_charstems
from _lodes_data_structure import all_stems
from _lodes_data_structure import all_jobtypes
from _lodes_data_structure import all_mxjobtypes
from _lodes_data_structure import all_segstems
from lodes_datautil import *
from lodes_fullloop import *
from lodes_mcmcsa_util import *

In [ ]:
arise_county_fips = {"johnson" : "20091",
                     "wyandotte" : "20209",
                     "finney" : "20055",
                     "ford" : "20057",
                     "seward" : "20175"}

In [ ]:
target_county = "ford"

In [ ]:
county_fips= arise_county_fips[target_county]

In [ ]:
stacked_df = obtain_lodes_county_loop([county_fips], 
                    ['2020'], 
                    outputfoldername = "output",
                    ods = all_ods,
                    segparts = all_segparts,
                    jobtypes = all_jobtypes,
                    mxjobtypes = all_mxjobtypes,
                    segstems = all_segstems,
                    blocklist = '')

In [ ]:
work_block_list = get_county_work_block_list(stacked_df, county_fips_code=county_fips, year='2020', od='od', seg='na')
home_block_list = get_county_home_block_list(stacked_df, county_fips_code=county_fips, year='2020', od='od', seg='na')
print("work block list: ", len(work_block_list))
print("home block list: ", len(home_block_list))

In [ ]:
# for block in work_block_list[0:10]:
#     out_of_state_rac_blocks_df = out_of_state_rac_blocks(work_block=block, 
#                             years= ['2020'], 
#                             outputfoldername = "output",
#                             stacked_df=stacked_df,
#                             segstems = ['SE','SI','SA'])
#     print(len(out_of_state_rac_blocks_df))

In [ ]:
seed_value = 1234
counter=1
for od in ["rac"]: #["wac", "rac"]:
    joblist_df = pd.DataFrame()
    if od== "wac":
        block_list = work_block_list
    elif od == "rac":
        block_list = home_block_list
    
    for block in block_list:
        seed_value += 1
        joblist = wac_rac_block_to_joblist(stacked_df = stacked_df, 
                                            block_fips= block ,
                                            years = ['2020'],
                                            seed_value = seed_value,
                                            outputfoldername = "output",
                                            od = od,
                                            reshape_vars = {'CE' : 'Earnings',
                                                                'CNS': 'IndustryCode',
                                                                'CA' : 'Age',
                                                                'CR' : 'Race',
                                                                'CT' : 'Ethnicity',
                                                                'CD' : 'Education',
                                                                'CS' : 'Sex'},
                                                            segstems = ['SE','SI','SA'])
        
        for (year, od), inner_dict in joblist.items():
            for key, df in inner_dict.items():
                df.to_csv(f'output2/{od}_{block}_joblist_{year}.csv', index=False)
                if counter == 1:
                    joblist_df = df.copy()  # Create a new DataFrame
                else:
                    joblist_df = pd.concat([joblist_df, df], ignore_index=True)   
        counter += 1
        print("*********************************************************************")

    joblist_df.to_csv(f'{od}_{target_county}_county_joblist_{year}.csv', index=False)
    # display(wac_joblist_df)